In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, balanced_accuracy_score
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight

from vctr.data.data_loader import get_data_with_features_and_labels

symbol = 'RNDR'
timeframe = '30m'

data = get_data_with_features_and_labels(
    symbol=symbol,
    timeframes=[timeframe],
    separate=False,
    label_args=(0.06, 0.0075),
)

data['label'][data['label'] == 1] = 0
data['label'][data['label'] == 2] = 1

# data['close'].vbt.overlay_with_heatmap(data['label']).show()

X, y = data.drop(['label'], axis=1), data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test = StandardScaler().fit_transform(X_train), StandardScaler().fit_transform(X_test)

# Calculate class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = dict(zip(np.unique(y_train), class_weights))

# CatBoost with class weights
catboost_clf = CatBoostClassifier(
    iterations=750, learning_rate=0.1, class_weights=class_weights_dict, verbose=False
)
catboost_clf.fit(X_train, y_train)
catboost_preds = catboost_clf.predict(X_test)

# Evaluate classifiers using balanced accuracy
print('CatBoost balanced accuracy: ', balanced_accuracy_score(y_test, catboost_preds))
print('\nCatBoost classification report:\n', classification_report(y_test, catboost_preds))

catboost_clf.save_model('sell_clf.model')


ReadTimeout: HTTPSConnectionPool(host='api.polygon.io', port=443): Read timed out. (read timeout=10)

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier
import vectorbtpro as vbt

from vctr.data.data_loader import get_data_with_features_and_labels
from vctr.utils.pf_stats import get_plot_and_pf, print_stats
import faulthandler

faulthandler.enable()

buy_clf = CatBoostClassifier().load_model('buy_clf.model')
sell_clf = CatBoostClassifier().load_model('sell_clf.model')

symbol = 'ADA'
timeframe = '15m'

data = get_data_with_features_and_labels(
    symbol=symbol,
    timeframe=timeframe,
    start='2023-01-01',
    separate=False,
    label_args=(0.03, 0.005),
)

X, y = data.drop(['label'], axis=1), data['label']
X = StandardScaler().fit_transform(X)

buy_preds = buy_clf.predict(X)
sell_preds = sell_clf.predict(X)

data = data[-len(sell_preds) :].copy()
data['buy_pred'] = pd.Series(buy_preds, index=data.index)
data['sell_pred'] = pd.Series(sell_preds, index=data.index)

pf = vbt.Portfolio.from_signals(
    data['close'],
    data['buy_pred'] == 1,
    data['sell_pred'] == 1,
    freq='15T',
    init_cash=10000,
    fees=0.0006,
    slippage=0.001,
    log=True,
)
# fig = pf.trades.plot()

# axis_args = dict(showgrid=True, gridwidth=0.5, gridcolor='rgba(255, 255, 255, 0.05)', griddash='dot')
# fig.update_xaxes(**axis_args)
# fig.update_yaxes(**axis_args)
# fig.update_layout(dict(width=1050, height=600))

# fig.data[0].line.width = 1
# fig.data[0].opacity = 0.75
# fig.show()
print_stats(pf)